## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,2020-08-23 20:08:22,64.38,-173.30,50.00,81,90,2.24,overcast clouds
1,1,Rikitea,PF,2020-08-23 20:03:58,-23.12,-134.97,70.09,84,79,22.08,broken clouds
2,2,Hobyo,SO,2020-08-23 20:05:00,5.35,48.53,75.97,77,60,24.00,broken clouds
3,3,Batagay,RU,2020-08-23 20:08:23,67.63,134.63,34.12,93,57,3.60,broken clouds
4,4,Nizhniy Kuranakh,RU,2020-08-23 20:08:23,58.84,125.49,39.52,78,0,1.48,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2020-08-23 20:03:58,-23.12,-134.97,70.09,84,79,22.08,broken clouds
2,2,Hobyo,SO,2020-08-23 20:05:00,5.35,48.53,75.97,77,60,24.00,broken clouds
5,5,Sorong,ID,2020-08-23 20:08:23,-0.88,131.25,77.25,87,81,7.40,broken clouds
7,7,Avarua,CK,2020-08-23 20:04:49,-21.21,-159.78,75.20,73,0,11.41,clear sky
8,8,Puerto Ayora,EC,2020-08-23 20:03:54,-0.74,-90.35,77.00,57,59,21.92,broken clouds
12,12,Polson,US,2020-08-23 20:08:24,47.69,-114.16,82.99,40,0,2.19,clear sky
13,13,Vaini,TO,2020-08-23 20:03:50,-21.20,-175.20,75.20,94,20,6.93,few clouds
15,15,Jamestown,US,2020-08-23 20:02:39,42.10,-79.24,82.00,69,75,5.82,broken clouds
18,18,Katsuura,JP,2020-08-23 20:05:10,35.13,140.30,77.00,94,75,8.05,shower rain
21,21,Butaritari,KI,2020-08-23 20:04:13,3.07,172.79,82.80,69,99,10.87,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                310
City                   310
Country                310
Date                   310
Lat                    310
Lng                    310
Max Temp               310
Humidity               310
Cloudiness             310
Wind Speed             310
Current Description    310
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.09,broken clouds,-23.12,-134.97,
2,Hobyo,SO,75.97,broken clouds,5.35,48.53,
5,Sorong,ID,77.25,broken clouds,-0.88,131.25,
7,Avarua,CK,75.20,clear sky,-21.21,-159.78,
8,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
12,Polson,US,82.99,clear sky,47.69,-114.16,
13,Vaini,TO,75.20,few clouds,-21.20,-175.20,
15,Jamestown,US,82.00,broken clouds,42.10,-79.24,
18,Katsuura,JP,77.00,shower rain,35.13,140.30,
21,Butaritari,KI,82.80,overcast clouds,3.07,172.79,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [9]:
# Print Data frame
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.09,broken clouds,-23.12,-134.97,Pension Maro'i
2,Hobyo,SO,75.97,broken clouds,5.35,48.53,Osmani Hotel
5,Sorong,ID,77.25,broken clouds,-0.88,131.25,The Waigo Hotel
7,Avarua,CK,75.20,clear sky,-21.21,-159.78,Paradise Inn
8,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
12,Polson,US,82.99,clear sky,47.69,-114.16,Americas Best Value Port Polson Inn
13,Vaini,TO,75.20,few clouds,-21.20,-175.20,Keleti Beach Resort
15,Jamestown,US,82.00,broken clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
18,Katsuura,JP,77.00,shower rain,35.13,140.30,Katsuura Hotel Mikazuki
21,Butaritari,KI,82.80,overcast clouds,3.07,172.79,Isles Sunset Lodge


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace = True)

hotel_df.dropna(subset = ['Hotel Name'], inplace=True)

hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.09,broken clouds,-23.12,-134.97,Pension Maro'i
2,Hobyo,SO,75.97,broken clouds,5.35,48.53,Osmani Hotel
5,Sorong,ID,77.25,broken clouds,-0.88,131.25,The Waigo Hotel
7,Avarua,CK,75.20,clear sky,-21.21,-159.78,Paradise Inn
8,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
12,Polson,US,82.99,clear sky,47.69,-114.16,Americas Best Value Port Polson Inn
13,Vaini,TO,75.20,few clouds,-21.20,-175.20,Keleti Beach Resort
15,Jamestown,US,82.00,broken clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
18,Katsuura,JP,77.00,shower rain,35.13,140.30,Katsuura Hotel Mikazuki
21,Butaritari,KI,82.80,overcast clouds,3.07,172.79,Isles Sunset Lodge


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 0.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))